In [13]:
#!pip install snowflake-connector-python==2.7.9

In [155]:
import snowflake
import snowflake.connector
import pandas as pd

In [156]:
# Configure Pandas to display all columns and rows
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)

# Suitable default display for floats
pd.options.display.float_format = '{:,.2f}'.format

%matplotlib inline
plt.rcParams["figure.figsize"] = (10,8)
sns.set_style("whitegrid")
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

NameError: name 'plt' is not defined

## Connect to database

In [157]:
conn = snowflake.connector.connect(user = 'SVC_BEERTECH_CANDIDATE',
                                   password = '!uFbtIPCq53Ia5Er08*Ruc&blBPEei', 
                                   account = 'abinbev_naz.east-us-2.azure',
                                   warehouse = 'DEV_ELT_ADF_US_XS',
                                   database = 'candidate_sandbox',
                                   schema = 'ds_source')

## Get sample data for EDA

## Find unique brewers in NY wholesaler

In [127]:
sql_sales = """
            select 
            prod.BREWER_VALUE,
            count(distinct s.product_key) as num_products
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            and store.GEOGRAPHY_KEY in (258891)
            
            group by 1
            order by num_products desc
            ;
            
           """

pd.read_sql(sql_sales, conn)

,BREWER_VALUE,NUM_PRODUCTS
0,ANHEUSER-BUSCH INBEV,381
1,MILLERCOORS,180
2,BOSTON BEER CO,175
3,FIFCO USA,155
4,DUVEL MOORTGAT USA,150
5,ARTISANAL BREWING VENTURES,128
6,KIRIN BREWERY CO LTD.,103
7,HEINEKEN USA INC,94
8,DIAGEO BEER COMPANY,80
9,FOUNDERS BREWING CO,67


## Find unique brewers in NY all wholesalers

In [128]:
sql_sales = """
            select 
            prod.BREWER_VALUE,
            count(distinct s.product_key) as num_products
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            
            group by 1
            order by num_products desc
            ;
            
           """

pd.read_sql(sql_sales, conn)

,BREWER_VALUE,NUM_PRODUCTS
0,ANHEUSER-BUSCH INBEV,1458
1,MILLERCOORS,695
2,BOSTON BEER CO,441
3,FIFCO USA,356
4,DUVEL MOORTGAT USA,306
5,HEINEKEN USA INC,259
6,ARTISANAL BREWING VENTURES,236
7,DIAGEO BEER COMPANY,221
8,KIRIN BREWERY CO LTD.,220
9,MARK ANTHONY BRANDS INC,205


## Find unique outlets in NY wholesaler

In [129]:
sql_sales = """
            select 
            store.OUTLET_NAME,
            count(distinct s.product_key) as num_products
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            and store.GEOGRAPHY_KEY in (258891)
            
            group by 1
            order by num_products desc
            ;
            
           """

pd.read_sql(sql_sales, conn)

,OUTLET_NAME,NUM_PRODUCTS
0,GROCERY,3711


## Find unique brewers in NY all wholesalers

In [130]:
sql_sales = """
            select 
            store.OUTLET_NAME,
            count(distinct s.product_key) as num_products
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            
            group by 1
            order by num_products desc
            ;
            
           """

pd.read_sql(sql_sales, conn)

,OUTLET_NAME,NUM_PRODUCTS
0,GROCERY,8289
1,CONVENIENCE,6260
2,LIQUOR,6247
3,DRUG,3464
4,MILITARY,2131
5,MASS MERCHANDISER,2047
6,CLUB,242


## For NY

In [201]:
# wholesalers from each outlet

sql_sales = """
            select
            store.OUTLET_NAME,
            store.GEOGRAPHY_KEY,
            prod.BREWER_VALUE,
            s.product_key,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            -- and store.OUTLET_NAME in ('CLUB')
            and prod.BREWER_VALUE in ('ANHEUSER-BUSCH INBEV')
            
            group by 1,2,3,4
            order by sum(UNIT_SALES) desc
            limit 20;
            
           """

ny = pd.read_sql(sql_sales, conn)
ny

,OUTLET_NAME,GEOGRAPHY_KEY,BREWER_VALUE,PRODUCT_KEY,TOTAL_SALES
0,CONVENIENCE,5446689,ANHEUSER-BUSCH INBEV,210284670,"166,760.00"
1,CONVENIENCE,627036,ANHEUSER-BUSCH INBEV,210281271,"123,439.00"
2,DRUG,39774,ANHEUSER-BUSCH INBEV,210281271,"94,280.00"
3,CONVENIENCE,5446216,ANHEUSER-BUSCH INBEV,210281273,"93,000.00"
4,CONVENIENCE,623824,ANHEUSER-BUSCH INBEV,210281271,"82,967.00"
5,CONVENIENCE,5446342,ANHEUSER-BUSCH INBEV,210281273,"76,827.00"
6,DRUG,39774,ANHEUSER-BUSCH INBEV,210281273,"75,618.00"
7,CONVENIENCE,5446689,ANHEUSER-BUSCH INBEV,210281273,"66,773.00"
8,DRUG,80499,ANHEUSER-BUSCH INBEV,210281267,"64,901.00"
9,CONVENIENCE,630626,ANHEUSER-BUSCH INBEV,210283096,"63,858.00"


In [205]:
# common products

sql_sales = """
            select
            store.GEOGRAPHY_KEY,
            s.product_key,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            
            and store.STATE_CODE = 'NY'
            and store.GEOGRAPHY_KEY in (5446689, 255718, 39774, 147449, 16568147, 26919438, 627036)
        
            -- and prod.BREWER_VALUE in ('ANHEUSER-BUSCH INBEV')
            and s.PRODUCT_KEY in (210283096)
            
            group by 1,2 
            order by sum(UNIT_SALES) desc
            limit 10;
            
           """

ny = pd.read_sql(sql_sales, conn)
ny

,GEOGRAPHY_KEY,PRODUCT_KEY,TOTAL_SALES
0,39774,210283096,"58,847.00"
1,5446689,210283096,"23,499.00"
2,627036,210283096,"18,757.00"
3,147449,210283096,"3,890.00"
4,26919438,210283096,106.00


## For TX

In [162]:
sql_sales = """
            select
            store.OUTLET_NAME,
            store.GEOGRAPHY_KEY,
            prod.BREWER_VALUE,
            s.product_key,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('TX')
            -- and store.OUTLET_NAME in ('CLUB')
            
            group by 1,2,3,4
            order by sum(UNIT_SALES) desc
            limit 10;
            
           """

pd.read_sql(sql_sales, conn)

,OUTLET_NAME,GEOGRAPHY_KEY,BREWER_VALUE,PRODUCT_KEY,TOTAL_SALES
0,CONVENIENCE,25546391,ANHEUSER-BUSCH INBEV,210281267,"148,639.00"
1,GROCERY,929720,MILLERCOORS,1409125,"143,963.00"
2,CONVENIENCE,432033,ANHEUSER-BUSCH INBEV,1400908,"129,028.00"
3,CONVENIENCE,25566575,ANHEUSER-BUSCH INBEV,210281267,"123,744.00"
4,CONVENIENCE,430650,ANHEUSER-BUSCH INBEV,1400908,"122,031.00"
5,GROCERY,506262,ANHEUSER-BUSCH INBEV,210284670,"121,396.00"
6,CONVENIENCE,25573907,ANHEUSER-BUSCH INBEV,210281267,"120,139.00"
7,CONVENIENCE,25573931,ANHEUSER-BUSCH INBEV,210281267,"115,096.00"
8,CONVENIENCE,25573923,ANHEUSER-BUSCH INBEV,210281267,"114,375.00"
9,GROCERY,176528,ANHEUSER-BUSCH INBEV,210280223,"113,556.00"


In [204]:
# common products

sql_sales = """
            select
            store.GEOGRAPHY_KEY,
            s.product_key,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            
            and store.STATE_CODE = 'TX'
            and store.GEOGRAPHY_KEY in (25546391, 255718, 26919130, 158338, 26953786, 126759, 432033 )
        
            -- and prod.BREWER_VALUE in ('ANHEUSER-BUSCH INBEV')
            and s.PRODUCT_KEY in (210283096)
            
            group by 1,2 
            order by sum(UNIT_SALES) desc
            limit 10;
            
           """

tx = pd.read_sql(sql_sales, conn)
tx

,GEOGRAPHY_KEY,PRODUCT_KEY,TOTAL_SALES
0,25546391,210283096,263.00
1,432033,210283096,2.00


### Now, load data for the selected 6 wholesalers and 10 products from NY

In [206]:
sql_sales_ny = """
            select
            s.*, 
            to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy') as weekly_date,
            store.*,
            prod.*
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            and store.GEOGRAPHY_KEY in (5446689, 255718, 39774, 147449, 16568147, 26919438, 432033, 627036 )
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221,
                                  210281267, 210280222, 210284670, 210281273, 210283096);
            
           """

ny_df = pd.read_sql(sql_sales_ny, conn)
ny_df.head()

,PRODUCT_KEY,GEOGRAPHY_KEY,TIME_KEY,UNIT_SALES,DOLLAR_SALES,VOLUME_SALES,WEEKLY_DATE,GEOGRAPHY_KEY,GEOGRAPHY_DESCRIPTION,WHOLESALER_ID_VALUE,RETAILER_NAME,CITY,STATE_CODE,ZIP_CODE,OUTLET_NAME,PRODUCT_KEY,PRODUCT_DESCRIPTION,AB_CATEGORY_STYLE_VALUE,AB_WAMP_VALUE,BEER_STYLE_VALUE,BRAND_VALUE,BREWER_VALUE,TYPE_VALUE
0,210280222,39774,2051,15.00,29.85,1.30,2018-12-23,39774,"RITE AID PHARMACY 568 : 50 S MAIN ST, JAMESTOW...",03142,RITE AID - RITE AID PHARMACY,JAMESTOWN,NY,14701,DRUG,210280222,**********,CLASSIC LAGER,CORE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER
1,210281267,39774,2051,36.00,71.64,3.10,2018-12-23,39774,"RITE AID PHARMACY 568 : 50 S MAIN ST, JAMESTOW...",03142,RITE AID - RITE AID PHARMACY,JAMESTOWN,NY,14701,DRUG,210281267,**********,EASY DRINKING,CORE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER
2,210283096,5446689,2051,296.00,298.32,25.70,2018-12-23,5446689,"SPEEDWAY SUPERAMERICA 7877 : 2121 FOREST AVE, ...",43100,7 ELEVEN - SPEEDWAY SUPERAMERICA,STATEN ISLAND,NY,10303,CONVENIENCE,210283096,**********,CLASSIC LAGER,VALUE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER
3,210280222,627036,2051,35.00,71.21,3.00,2018-12-23,627036,7 ELEVEN/NORTH ATLANTIC DIV 35205 : 518 N MAIN...,03142,7-ELEVEN - 7 ELEVEN/NORTH ATLANTIC DIV,JAMESTOWN,NY,14701,CONVENIENCE,210280222,**********,CLASSIC LAGER,CORE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER
4,210283096,147449,2051,85.00,126.65,7.40,2018-12-23,147449,"K MART 7749 : 250 W 34TH ST, NEW YORK, NY",43100,KMART - K MART,NEW YORK,NY,10119,MASS MERCHANDISER,210283096,**********,CLASSIC LAGER,VALUE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER


In [222]:
ny_df.shape

(8599, 23)

### Now, load data for the selected 6 wholesalers and 10 products from TX

In [218]:
sql_sales_tx = """
            select
            s.*, 
            to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy') as weekly_date,
            store.*,
            prod.*
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('TX')
            and store.GEOGRAPHY_KEY in (25546391, 255718, 26919130, 158338, 26953786, 126759, 432033 )
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221,
                                  210281267, 210280222, 210284670, 210281273, 210283096);
            
           """

tx_df = pd.read_sql(sql_sales_tx, conn)
tx_df.head()

,PRODUCT_KEY,GEOGRAPHY_KEY,TIME_KEY,UNIT_SALES,DOLLAR_SALES,VOLUME_SALES,WEEKLY_DATE,GEOGRAPHY_KEY,GEOGRAPHY_DESCRIPTION,WHOLESALER_ID_VALUE,RETAILER_NAME,CITY,STATE_CODE,ZIP_CODE,OUTLET_NAME,PRODUCT_KEY,PRODUCT_DESCRIPTION,AB_CATEGORY_STYLE_VALUE,AB_WAMP_VALUE,BEER_STYLE_VALUE,BRAND_VALUE,BREWER_VALUE,TYPE_VALUE
0,609521221,255718,2051,18.00,233.46,9.00,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,H-E-B - H-E-B PLUS,LEANDER,TX,78641,GROCERY,609521221,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
1,1412422,126759,2051,3.00,41.97,1.50,2018-12-23,126759,"TARGET PFRESH 849 : 6001 W GATEWAY BLVD, EL PA...",44223,TARGET - TARGET PFRESH,EL PASO,TX,79925,MASS MERCHANDISER,1412422,**********,EASY DRINKING,PREMIUM,TROPICAL LAGER,**********,CONSTELLATION BRANDS,BEER
2,210284670,255718,2051,6.00,10.02,0.50,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,H-E-B - H-E-B PLUS,LEANDER,TX,78641,GROCERY,210284670,**********,CLASSIC LAGER,VALUE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER
3,210281267,255718,2051,24.00,49.68,2.10,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,H-E-B - H-E-B PLUS,LEANDER,TX,78641,GROCERY,210281267,**********,EASY DRINKING,CORE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER
4,210280222,255718,2051,8.00,16.55,0.70,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,H-E-B - H-E-B PLUS,LEANDER,TX,78641,GROCERY,210280222,**********,CLASSIC LAGER,CORE,AMERICAN LAGER,**********,ANHEUSER-BUSCH INBEV,BEER


In [221]:
tx_df.shape

(7758, 23)

## Get data for modelling

## Find two wholesalers in NY & TX with one of top 10 highest unit sales since 2018

In [67]:
sql_sales = """
            select store.GEOGRAPHY_KEY,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            
            group by 1
            order by sum(UNIT_SALES) desc
            limit 10;
            
           """

pd.read_sql(sql_sales, conn)

,GEOGRAPHY_KEY,TOTAL_SALES
0,16568147,2.198203e+06
1,252791,1.584977e+06
2,662205,1.565420e+06
3,233057,1.485250e+06
4,134584,1.400589e+06
5,120262,1.388554e+06
6,199504,1.374629e+06
7,181530,1.238046e+06
8,258891,1.192355e+06
9,32596,1.167116e+06


- **TX - 255718** 
- **NY - 258891** 

### Find five products in NY & TX 

In [121]:
# 5 popular products in NY

sql_sales = """
            select s.PRODUCT_KEY,
            -- prod.BREWER_VALUE,
            -- store.OUTLET_NAME,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            and store.GEOGRAPHY_KEY in (258891)
            
            group by 1,2
            order by sum(UNIT_SALES) desc
            limit 15;
            
           """

pd.read_sql(sql_sales, conn)

,PRODUCT_KEY,OUTLET_NAME,TOTAL_SALES
0,36849957,GROCERY,52794.0
1,680031864,GROCERY,19393.0
2,28488051,GROCERY,16881.0
3,659550424,GROCERY,11902.0
4,1413054,GROCERY,11049.0
5,1321133691,GROCERY,7513.0
6,210281273,GROCERY,7083.0
7,1400451,GROCERY,6853.0
8,154157221,GROCERY,6832.0
9,609521221,GROCERY,6028.0


In [89]:
# in Texas that also sell the popular products from NY

sql_sales = """
            select s.PRODUCT_KEY,
            sum(UNIT_SALES) as total_sales
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('TX')
            and store.GEOGRAPHY_KEY in (255718)
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221)
            
            group by 1
            order by sum(UNIT_SALES) desc;
            
           """

pd.read_sql(sql_sales, conn)

,PRODUCT_KEY,TOTAL_SALES
0,659550424,19413.0
1,1412422,11409.0
2,1321133691,6040.0
3,609521221,4153.0
4,262793636,1274.0


## Now, load data for the selected 2 wholesalers and 5 products from TX

In [102]:
sql_sales_tx = """
            select
            s.*, 
            to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy') as weekly_date,
            store.*,
            prod.*
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('TX')
            and store.GEOGRAPHY_KEY in (255718)
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221);
            
           """

tx_df = pd.read_sql(sql_sales_tx, conn)
tx_df.head()

,PRODUCT_KEY,GEOGRAPHY_KEY,TIME_KEY,UNIT_SALES,DOLLAR_SALES,VOLUME_SALES,WEEKLY_DATE,GEOGRAPHY_KEY,GEOGRAPHY_DESCRIPTION,WHOLESALER_ID_VALUE,...,ZIP_CODE,OUTLET_NAME,PRODUCT_KEY,PRODUCT_DESCRIPTION,AB_CATEGORY_STYLE_VALUE,AB_WAMP_VALUE,BEER_STYLE_VALUE,BRAND_VALUE,BREWER_VALUE,TYPE_VALUE
0,1412422,255718,2051,47.0,694.66,23.5,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,1412422,**********,EASY DRINKING,PREMIUM,TROPICAL LAGER,**********,CONSTELLATION BRANDS,BEER
1,609521221,255718,2051,18.0,233.46,9.0,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,609521221,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
2,1321133691,255718,2051,27.0,350.19,13.5,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,1321133691,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
3,659550424,255718,2051,77.0,998.69,38.5,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,659550424,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,MARK ANTHONY BRANDS INC,MALT BEVERAGE
4,262793636,255718,2051,7.0,111.93,4.4,2018-12-23,255718,"H-E-B PLUS 592 : 651 N US HIGHWAY 183, LEANDER...",34206,...,78641,GROCERY,262793636,**********,CRAFT & STYLES,PREMIUM,IPA,**********,FOUNDERS BREWING CO,BEER


## Now, load data for the selected wholesaler and 5 products from NY

In [103]:
sql_sales_ny = """
            select
            s.*, 
            to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy') as weekly_date,
            store.*,
            prod.*
            from candidate_sandbox.ds_source.abi_ddm_sales s
            join candidate_sandbox.ds_source.abi_ddm_week w
                on s.TIME_KEY = w.TIME_KEY
            join candidate_sandbox.ds_source.abi_ddm_stores store
                on s.GEOGRAPHY_KEY = store.GEOGRAPHY_KEY
            join candidate_sandbox.ds_source.abi_ddm_products prod
                on s.PRODUCT_KEY = prod.PRODUCT_KEY
            where year(to_date(substr(w.TIME_DESCRIPTION, 12)::varchar, 'mm-dd-yy')) >= 2018
            and store.STATE_CODE in ('NY')
            and store.GEOGRAPHY_KEY in (258891)
            and s.PRODUCT_KEY in (1321133691, 659550424, 1412422, 262793636,  609521221);
            
           """

ny_df = pd.read_sql(sql_sales_ny, conn)
ny_df.head()

,PRODUCT_KEY,GEOGRAPHY_KEY,TIME_KEY,UNIT_SALES,DOLLAR_SALES,VOLUME_SALES,WEEKLY_DATE,GEOGRAPHY_KEY,GEOGRAPHY_DESCRIPTION,WHOLESALER_ID_VALUE,...,ZIP_CODE,OUTLET_NAME,PRODUCT_KEY,PRODUCT_DESCRIPTION,AB_CATEGORY_STYLE_VALUE,AB_WAMP_VALUE,BEER_STYLE_VALUE,BRAND_VALUE,BREWER_VALUE,TYPE_VALUE
0,1412422,258891,2051,22.0,318.78,11.0,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,1412422,**********,EASY DRINKING,PREMIUM,TROPICAL LAGER,**********,CONSTELLATION BRANDS,BEER
1,609521221,258891,2051,32.0,479.68,16.0,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,609521221,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
2,1321133691,258891,2051,33.0,494.67,16.5,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,1321133691,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,BOSTON BEER CO,MALT BEVERAGE
3,659550424,258891,2051,30.0,448.70,15.0,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,659550424,**********,HARD BEVERAGES (BEER),BEYOND BEER,VARIETY PACK/SAMPLER,**********,MARK ANTHONY BRANDS INC,MALT BEVERAGE
4,262793636,258891,2051,22.0,373.78,13.8,2018-12-23,258891,"WEGMANS 91 : 601 AMHERST ST, BUFFALO, NY",33107,...,14207,GROCERY,262793636,**********,CRAFT & STYLES,PREMIUM,IPA,**********,FOUNDERS BREWING CO,BEER


In [110]:
ny_df.shape, tx_df.shape

((1292, 23), (1242, 23))

## Cache data 

In [223]:
ny_df.to_csv('data/ny_df.csv', index=False)
tx_df.to_csv('data/tx_df.csv', index=False)